In [1]:
!pip install -q tensorflow

import numpy as np
import tensorflow as tf
import random as rn

In [2]:
import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(37)
rn.seed(1254)
tf.random.set_seed(89)

In [3]:
from tensorflow.python.keras import backend as K

# force tensorflow to use a single thread
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),config=config)
K.set_session(sess)

In [4]:
## Imports libs
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

from tensorflow.keras import layers
from keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam

Segmentation Models: using `tf.keras` framework.


Using TensorFlow backend.


In [5]:
!pip install -U -q segmentation-models
!pip install -q keras
!pip install -q tensorflow-estimator

import pandas as pd
import cv2
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image
from random import shuffle
from tqdm import tqdm
import pandas as pd
from sklearn.utils import class_weight

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

from tensorflow.keras.models import Model,Sequential, load_model,model_from_json
from tensorflow.keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,vgg16,resnet_v2,resnet
#from tensorflow.keras.utils.generic_utils import CustomObjectScope
from tensorflow.keras.layers import Flatten, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from tensorflow.keras.metrics import top_k_categorical_accuracy
from sklearn.metrics import confusion_matrix

In [6]:
BATCH_SIZE=80
INPUT_SIZE = (224, 224)

In [7]:
model=load_model(r'C:\Users\HP\FER_SL\models\affectnet_emotions\mobilenet_7.h5')
model.summary()
net_description='mobilenet_7'
def mobilenet_preprocess_input(x,**kwargs):
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x
preprocessing_function=mobilenet_preprocess_input
        
BATCH_SIZE=80
INPUT_SIZE = (224, 224)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [8]:
net_description='FT_IP_HF_'+net_description

In [9]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function,horizontal_flip = True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocessing_function)

In [10]:
target_size = (224, 224)

training_set = train_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\train_files_cropped', target_size=target_size,
                                                 batch_size=BATCH_SIZE, class_mode='categorical')

test_set = test_datagen.flow_from_directory(r'C:\Users\HP\FER_SL\sl_data\test_files_cropped', target_size=target_size,
                                            batch_size=BATCH_SIZE, class_mode='categorical')

Found 2024 images belonging to 7 classes.
Found 507 images belonging to 7 classes.


# Downloading the previous model

In [11]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [12]:
def count_params(model):
    non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
    trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
    return {'non_trainable_params': non_trainable_params, 'trainable_params': trainable_params}

In [13]:
params = count_params(model)
assert params['non_trainable_params'] == 21888
assert params['trainable_params'] == 3471175

# Removing the last layer

In [14]:
new_model = Sequential()
for layer in model.layers[:-1]:
    new_model.add(layer)

# Freezing the previous layers

In [15]:
for layer in new_model.layers:
    layer.trainable = False

# Adding a new dense layer

In [16]:
print(model.input)

KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")


In [17]:
new_model.add(Dense(units=7, input_shape=(224,224), activation='softmax'))
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0

# Training only the last layer

In [18]:
params = count_params(new_model)
assert params['non_trainable_params'] == 3491264
# only the parameters in the last layer are trainable
assert params['trainable_params'] == 1799

In [19]:
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(net_description)
nb_train_samples = 2024
nb_validation_samples = 507

mc = ModelCheckpoint(net_description+'.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_accuracy',patience=2)
FIRST_EPOCHS=3

hist1=new_model.fit(training_set, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=FIRST_EPOCHS, verbose=1, 
                    initial_epoch=0, callbacks=[mc, es], validation_data=test_set, validation_steps=nb_validation_samples // BATCH_SIZE)

FT_IP_HF_mobilenet_7
Epoch 1/3
25/25 [==============================] - 85s 3s/step - loss: 2.3182 - accuracy: 0.1116 - val_loss: 1.9227 - val_accuracy: 0.1646

Epoch 00001: val_accuracy improved from -inf to 0.16458, saving model to FT_IP_HF_mobilenet_7.h5
Epoch 2/3
25/25 [==============================] - 78s 3s/step - loss: 1.7548 - accuracy: 0.3117 - val_loss: 1.5976 - val_accuracy: 0.3875

Epoch 00002: val_accuracy improved from 0.16458 to 0.38750, saving model to FT_IP_HF_mobilenet_7.h5
Epoch 3/3
25/25 [==============================] - 78s 3s/step - loss: 1.5657 - accuracy: 0.4285 - val_loss: 1.4368 - val_accuracy: 0.4729

Epoch 00003: val_accuracy improved from 0.38750 to 0.47292, saving model to FT_IP_HF_mobilenet_7.h5


# Re-training the model with all the layers

In [20]:
start_epoch=len(hist1.history['loss'])

for layer in new_model.layers:
    layer.trainable = True
    
new_model.summary()

params = count_params(new_model)
assert params['non_trainable_params'] == 21888
# only the parameters in the last layer are trainable
assert params['trainable_params'] == 3471175 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0

In [21]:
new_model.load_weights(net_description+'.h5')
new_model.compile(optimizer=Adam(learning_rate=1e-4,decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
SECOND_EPOCHS=start_epoch+5

mc = ModelCheckpoint(net_description+'_ft.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
#es=EarlyStopping(monitor='val_accuracy',patience=2 )

print('All tuning')
hist2=new_model.fit(training_set, steps_per_epoch=training_set.samples//BATCH_SIZE, epochs=SECOND_EPOCHS, verbose=1, 
                    initial_epoch=start_epoch, validation_data=test_set, validation_steps=test_set.samples // BATCH_SIZE, callbacks=[mc])

All tuning
Epoch 4/8
25/25 [==============================] - 366s 14s/step - loss: 1.4023 - accuracy: 0.5082 - val_loss: 1.3269 - val_accuracy: 0.4563

Epoch 00004: val_accuracy improved from -inf to 0.45625, saving model to FT_IP_HF_mobilenet_7_ft.h5
Epoch 5/8
25/25 [==============================] - 350s 14s/step - loss: 1.1129 - accuracy: 0.6091 - val_loss: 1.1511 - val_accuracy: 0.5521

Epoch 00005: val_accuracy improved from 0.45625 to 0.55208, saving model to FT_IP_HF_mobilenet_7_ft.h5
Epoch 6/8
25/25 [==============================] - 347s 14s/step - loss: 0.9163 - accuracy: 0.6826 - val_loss: 1.0144 - val_accuracy: 0.6292

Epoch 00006: val_accuracy improved from 0.55208 to 0.62917, saving model to FT_IP_HF_mobilenet_7_ft.h5
Epoch 7/8
25/25 [==============================] - 366s 15s/step - loss: 0.7783 - accuracy: 0.7260 - val_loss: 0.9183 - val_accuracy: 0.6687

Epoch 00007: val_accuracy improved from 0.62917 to 0.66875, saving model to FT_IP_HF_mobilenet_7_ft.h5
Epoch 8/8
25

In [23]:
new_model.load_weights(net_description+'_ft.h5')

In [24]:
new_model.compile(optimizer=SGD(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
#new_model.summary()
mc = ModelCheckpoint(net_description+'_ft_sgd.h5', monitor='val_accuracy', verbose=1, save_best_only=True)
second_epoch=start_epoch+len(hist2.history['loss'])
THIRD_EPOCHS=2+second_epoch

print('ALL SGD tuning')
hist3=new_model.fit_generator(training_set, steps_per_epoch=training_set.samples//BATCH_SIZE, epochs=THIRD_EPOCHS, verbose=1, 
                    initial_epoch=second_epoch, validation_data=test_set, validation_steps=test_set.samples // BATCH_SIZE,callbacks=[mc])

ALL SGD tuning


c:\program files\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 9/10
25/25 [==============================] - 344s 13s/step - loss: 0.5815 - accuracy: 0.8056 - val_loss: 0.8959 - val_accuracy: 0.6708

Epoch 00009: val_accuracy improved from -inf to 0.67083, saving model to FT_IP_HF_mobilenet_7_ft_sgd.h5
Epoch 10/10
25/25 [==============================] - 350s 14s/step - loss: 0.5961 - accuracy: 0.7906 - val_loss: 0.8785 - val_accuracy: 0.6729

Epoch 00010: val_accuracy improved from 0.67083 to 0.67292, saving model to FT_IP_HF_mobilenet_7_ft_sgd.h5


In [25]:
USE_PREPROCESSING=True
print(INPUT_SIZE)

(224, 224)


In [26]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [27]:
import numpy as np
import cv2
y_val=np.array(test_set.labels)
y_scores_val=[]
#new_model.load_weights(net_description+'_ft.h5')
for filepath in log_progress(test_set.filepaths):
    img=cv2.imread(filepath)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img=cv2.resize(img,INPUT_SIZE)
    if USE_PREPROCESSING:
        inp = preprocessing_function(np.expand_dims(img, axis=0).astype(np.float32))
    else:
        inp=img.astype(np.float32)
        inp[..., 0] -= 103.939
        inp[..., 1] -= 116.779
        inp[..., 2] -= 123.68
        inp = np.expand_dims(inp, axis=0)

    scores=new_model.predict(inp)
    print(scores)
    y_scores_val.append(scores)

y_scores_val=np.concatenate(y_scores_val, axis=0)
print(y_scores_val.shape,y_val.shape)

[[0.20755374 0.00215776 0.0891303  0.00614771 0.07824195 0.03184021
  0.5849283 ]]
[[0.3508896  0.016313   0.13961881 0.00884865 0.09115884 0.1460466
  0.24712446]]
[[0.438738   0.01297243 0.11739361 0.00974642 0.09158893 0.16962749
  0.15993313]]
[[8.7873536e-01 7.1110241e-02 4.2892925e-02 1.0920295e-03 1.8165681e-03
  5.5946253e-04 3.7934114e-03]]
[[0.7242806  0.05248533 0.11165132 0.00358418 0.01122878 0.00224921
  0.09452058]]
[[0.8356098  0.03757203 0.08003913 0.00536524 0.00930649 0.00166765
  0.0304397 ]]
[[0.74170357 0.08715679 0.09922672 0.01239    0.01971378 0.00178308
  0.0380261 ]]
[[6.3299680e-01 1.6363008e-02 1.8701345e-01 4.6148766e-03 9.1266697e-03
  2.9992868e-04 1.4958522e-01]]
[[7.6668704e-01 3.6807399e-02 1.1669047e-01 4.2714719e-03 1.0402370e-02
  5.8097194e-04 6.4560257e-02]]
[[0.7167833  0.01896326 0.15478127 0.0076664  0.00948921 0.00080952
  0.0915071 ]]
[[0.5848251  0.05101702 0.24155273 0.01154008 0.00791011 0.00083256
  0.10232241]]
[[0.7344094  0.19464158 0

[[0.02862913 0.01600586 0.66402924 0.00293753 0.00898681 0.0104525
  0.26895884]]
[[0.19236988 0.24825877 0.43501553 0.00902431 0.02242513 0.02929481
  0.0636116 ]]
[[0.68445027 0.13227622 0.0751087  0.02208369 0.03307472 0.00916671
  0.04383966]]
[[9.6097958e-01 4.5918734e-03 8.0688280e-04 1.0520003e-03 2.1695851e-03
  2.8870841e-02 1.5292012e-03]]
[[9.6651131e-01 7.4984846e-03 5.1691942e-04 1.1741001e-03 2.0066160e-03
  2.1074245e-02 1.2183099e-03]]
[[9.4819152e-01 2.4002593e-02 6.8813730e-03 6.4640353e-04 8.5766305e-04
  1.5014048e-02 4.4063218e-03]]
[[0.8949855  0.03604858 0.0072444  0.0012776  0.00260996 0.05000344
  0.00783039]]
[[0.9018529  0.03145041 0.01204929 0.00206097 0.00384331 0.03549253
  0.01325053]]
[[9.3174136e-01 1.8122537e-02 6.8185008e-03 9.2448556e-04 2.0698945e-03
  3.1242652e-02 9.0805730e-03]]
[[0.90894294 0.02843284 0.00833899 0.00117218 0.00236009 0.04059942
  0.01015356]]
[[9.3098038e-01 1.2821298e-02 1.0913168e-02 8.4504153e-04 2.1173924e-03
  2.8330976e-02

[[0.04754388 0.7590774  0.1767345  0.00371372 0.00487108 0.0051245
  0.00293486]]
[[0.06607404 0.22336973 0.5490701  0.03603786 0.03029996 0.05968936
  0.03545894]]
[[0.09570675 0.13632776 0.5461694  0.04206334 0.05387717 0.06124743
  0.06460814]]
[[0.22329567 0.25527412 0.41365805 0.01735622 0.01684807 0.04346837
  0.03009944]]
[[0.19940169 0.3710321  0.29095212 0.0637435  0.02820796 0.01787225
  0.0287905 ]]
[[0.1449238  0.22310287 0.48841643 0.03166855 0.02109261 0.03664097
  0.05415475]]
[[0.24011277 0.69995445 0.02657788 0.0131742  0.01184406 0.00647964
  0.00185706]]
[[0.21133627 0.66510797 0.05552747 0.00188307 0.00510542 0.04752984
  0.01350996]]
[[9.8716968e-04 1.4142581e-06 2.1565937e-02 3.1273157e-05 1.9153990e-04
  2.8786935e-06 9.7721976e-01]]
[[0.01258892 0.00632783 0.09815515 0.00668266 0.138743   0.140983
  0.59651953]]
[[0.0137355  0.00657448 0.12736641 0.00875798 0.1939628  0.16942321
  0.4801796 ]]
[[1.0982451e-02 3.8622224e-04 7.5694062e-02 9.8792957e-03 1.7018357e-

[[0.45228106 0.19934368 0.01129872 0.13469923 0.11339063 0.08581247
  0.00317423]]
[[0.72394335 0.10756893 0.00582383 0.018948   0.0367732  0.10223465
  0.00470803]]
[[7.4034664e-03 4.2972071e-04 4.8961002e-01 1.1215507e-02 3.5391629e-02
  1.3288208e-02 4.4266149e-01]]
[[6.8099368e-03 2.7154258e-04 3.0077723e-01 1.4696434e-02 7.7202879e-02
  3.6871336e-02 5.6337070e-01]]
[[0.6536204  0.01046403 0.00111185 0.00155979 0.05556678 0.27348605
  0.00419095]]
[[0.782174   0.01466925 0.01726583 0.00353775 0.04281358 0.0813577
  0.058182  ]]
[[5.0388480e-04 2.2243030e-04 3.3933777e-02 1.2897389e-04 1.5343257e-03
  9.5441920e-01 9.2573874e-03]]
[[2.2270445e-04 6.3029525e-05 3.2034121e-03 3.6053458e-05 1.2088575e-03
  9.9392176e-01 1.3442201e-03]]
[[5.4143672e-04 2.2078709e-04 2.1361657e-03 7.9442048e-05 1.3388554e-03
  9.9481529e-01 8.6799986e-04]]
[[2.3511340e-04 1.0272417e-03 6.3749653e-04 5.5838123e-06 8.0593411e-05
  9.9776649e-01 2.4737007e-04]]
[[3.2685552e-04 4.5793466e-05 3.2340726e-04 1

[[6.9669408e-01 1.8098421e-03 2.4321750e-02 1.7285904e-04 2.1236371e-03
  7.7359006e-03 2.6714191e-01]]
[[0.05310818 0.02922646 0.34532726 0.01531114 0.05449487 0.27713478
  0.2253974 ]]
[[0.03866263 0.03834888 0.22801128 0.01635145 0.09075085 0.33597487
  0.2519    ]]
[[0.03312685 0.01844285 0.25092566 0.03394598 0.14427778 0.08751342
  0.43176752]]
[[0.03000868 0.0625312  0.27275524 0.02127714 0.0871993  0.28929842
  0.23693003]]
[[2.4819892e-02 6.8029301e-04 1.5118071e-01 3.8564866e-03 1.4034899e-02
  2.6052804e-03 8.0282241e-01]]
[[0.09483878 0.0413244  0.20086423 0.01797108 0.07774643 0.39437696
  0.17287813]]
[[0.1996848  0.02872517 0.01995942 0.04406951 0.2928271  0.34914246
  0.06559151]]
[[0.01499615 0.00555794 0.12328564 0.00402153 0.15367702 0.5672701
  0.13119166]]
[[0.07956649 0.00759283 0.14401016 0.00528795 0.1835247  0.24144831
  0.33856952]]
[[2.0708935e-03 1.0374998e-04 1.5513679e-01 1.2327520e-03 3.2574043e-03
  3.0583021e-04 8.3789253e-01]]
[[0.01195111 0.00195691 0

[[5.1914692e-05 1.0737228e-05 7.4432730e-03 2.9869116e-04 5.3151418e-03
  2.5226592e-04 9.8662800e-01]]
[[7.5286334e-05 1.5962431e-05 3.7642953e-03 2.8873797e-04 6.6096033e-03
  1.5055019e-04 9.8909563e-01]]
[[7.0970716e-05 9.9228591e-06 4.4975369e-03 2.9161692e-04 5.2220551e-03
  1.2213225e-04 9.8978567e-01]]
[[6.1093793e-05 1.0709004e-05 4.7072186e-03 2.6314970e-04 8.5603185e-03
  2.7180038e-04 9.8612565e-01]]
[[7.1504706e-05 9.9782983e-06 3.7641027e-03 2.3284568e-04 8.7599466e-03
  2.9389994e-04 9.8686767e-01]]
[[1.0851251e-04 1.6418788e-05 3.3031274e-03 3.1539288e-04 5.0392193e-03
  2.9123490e-04 9.9092603e-01]]
[[5.0103401e-05 1.4304983e-05 6.2688296e-03 3.1118529e-04 7.9026939e-03
  3.3305155e-04 9.8511988e-01]]
[[0.01363287 0.00648106 0.18159145 0.00403026 0.03832638 0.6287755
  0.12716256]]
[[0.004988   0.01361788 0.34176028 0.01144574 0.04363725 0.05667134
  0.5278795 ]]
[[0.0017219  0.00836272 0.03968794 0.11022031 0.09070518 0.01812869
  0.7311732 ]]
[[0.00316145 0.01105187 

In [28]:
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}

y_pred=np.argmax(y_scores_val,axis=1)
acc=100.0*(y_val==y_pred).sum()/len(y_val)
print('Accuracy:',acc)

y_train=np.array(training_set.labels)

for i in range(y_scores_val.shape[1]):
    _val_acc=(y_pred[y_val==i]==i).sum()/(y_val==i).sum()
    print('%s %d/%d acc: %f' %(idx_to_class[i],(y_train==i).sum(),(y_val==i).sum(),100*_val_acc))

Accuracy: 67.0611439842209
Anger 371/132 acc: 81.818182
Disgust 156/31 acc: 61.290323
Fear 251/57 acc: 40.350877
Happiness 160/19 acc: 68.421053
Neutral 153/42 acc: 33.333333
Sadness 290/59 acc: 50.847458
Surprise 643/167 acc: 79.640719


In [29]:
from sklearn.metrics import classification_report

target_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sadness', 'Surprise']
print(classification_report(y_val, y_pred, target_names=target_names))

              precision    recall  f1-score   support

       Anger       0.77      0.82      0.79       132
     Disgust       0.63      0.61      0.62        31
        Fear       0.52      0.40      0.46        57
       Happy       0.42      0.68      0.52        19
     Neutral       0.54      0.33      0.41        42
     Sadness       0.45      0.51      0.48        59
    Surprise       0.79      0.80      0.79       167

    accuracy                           0.67       507
   macro avg       0.59      0.59      0.58       507
weighted avg       0.67      0.67      0.67       507



# Making a single prediction

In [ ]:
from facial_analysis import FacialImageProcessing
imgProcessing=FacialImageProcessing(False)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential, load_model,model_from_json

my_model = load_model(r"C:\Users\HP\SLR\1src\sl_mobilenet_7_ft_sgd.h5") 
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = face_mesh.process(image)
        
        img = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        #cv2.imshow('MediaPipe FaceMesh', img)
        
        INPUT_SIZE = (224, 224)
        idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
        #filename = r'C:\Users\HP\SLR\test_files_1_sorted\07January_2010_Thursday_tagesschau_default-5.avi_pid0_fn000045-0.png'

        #img = plt.imread(filename)
        img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        img=cv2.resize(img,INPUT_SIZE)
        #frame_bgr=cv2.imread(filename)
        #frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(frame)
        bounding_boxes, points = imgProcessing.detect_faces(frame)
        points = points.T
        for bbox,p in zip(bounding_boxes, points):
            box = bbox.astype(np.int)
            x1,y1,x2,y2=box[0:4]    
            face_img=frame[y1:y2,x1:x2,:]

            face_img=cv2.resize(face_img,INPUT_SIZE)

            #plt.imshow(face_img)

            inp=face_img.astype(np.float32)
            inp[..., 0] -= 103.939
            inp[..., 1] -= 116.779
            inp[..., 2] -= 123.68
            inp = np.expand_dims(inp, axis=0)
            scores=my_model.predict(inp)
            #plt.title(idx_to_class[np.argmax(scores)])
            
            cv2.putText(frame, idx_to_class[np.argmax(scores)], (0, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 192, 203), 2)
            
            cv2.imshow('video', frame)
            
            #print(idx_to_class[np.argmax(scores)])

            #print(scores)

        
        if cv2.waitKey(5) & 0xFF == 27:
              break

cap.release()

In [ ]:
# For image 02August_2010_Monday_heute_default-7.avi_pid0_fn000008-0.png

INPUT_SIZE = (224, 224)
idx_to_class={0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
#filename = r'C:\Users\HP\SLR\test_files_1_sorted\07January_2010_Thursday_tagesschau_default-5.avi_pid0_fn000045-0.png'

#img = plt.imread(filename)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img=cv2.resize(img,INPUT_SIZE)
#frame_bgr=cv2.imread(filename)
frame_bgr=cv2.imread(img)
frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
plt.imshow(frame)
bounding_boxes, points = imgProcessing.detect_faces(frame)
points = points.T
for bbox,p in zip(bounding_boxes, points):
    box = bbox.astype(np.int)
    x1,y1,x2,y2=box[0:4]    
    face_img=frame[y1:y2,x1:x2,:]

    face_img=cv2.resize(face_img,INPUT_SIZE)

    plt.imshow(face_img)

    inp=face_img.astype(np.float32)
    inp[..., 0] -= 103.939
    inp[..., 1] -= 116.779
    inp[..., 2] -= 123.68
    inp = np.expand_dims(inp, axis=0)
    scores=new_model.predict(inp)
    plt.title(idx_to_class[np.argmax(scores)])

    print(scores)